In [3]:
import tkinter as tk
from tkinter import Y, simpledialog, messagebox
from datetime import datetime
import calendar
import webbrowser
import os
import random
import urllib.parse 

try:
    from PIL import Image, ImageTk
    PIL_AVAILABLE = True
except Exception:
    PIL_AVAILABLE = False

# ---------------- SETTINGS ----------------
NEON_COLORS = ["#39FF14", "#FF073A", "#00F5FF", "#FFFF00", "#DA00FF", "#00FFFF"]
MATRIX_CHARS = "0123456789ABCDEFGJHIKOMPZ"
SPEED = 1 # Terminal speed in ms (lower -> faster)
REMINDERS = {5: ["Doctor Appointment at 2 pm"], 12: ["Project Meeting", "Submit Report"], 25: ["Christmas Day 🎄"]}
BG_IMAGE_PATH = "/Users/FreddyDirectory/Picture/GS.PNG" # Path to gold skull image on your system   
BG = "#000000"

# ---------------- MAIN WINDOW ----------------
root = tk.Tk()
root.title("Swiss Python Automate Dashboard")
root.geometry("1200x700")
root.configure(bg=BG)
fullscreen = False

def toggle_fullscreen():
    global fullscreen
    fullscreen = not fullscreen
    root.attributes("-fullscreen", fullscreen)

# ---------------- LEFT FRAME (Calendar + Reminders) ----------------
class NeonCalendarWidget:
    def __init__(self, parent, month=12, year=2025):
        self.parent = parent
        self.real_today = datetime.today().date()
        self.current_month = month
        self.current_year = year

        # Main left container
        self.left_frame = tk.Frame(parent, bg=BG, width=320)
        self.left_frame.pack(side=tk.LEFT, fill=tk.Y, padx=6, pady=6)

        # UI elements placeholders
        self.today_label = None

        # Build UI
        self.create_calendar_section()
        self.create_reminder_section()

        # Start color cycle for today's label
        self._cycle_today_color()

    def _cycle_today_color(self):
        # Animate today's label with neon colors if it exists
        if self.today_label is not None:
            color = random.choice(NEON_COLORS)
            try:
                self.today_label.config(fg=color)
            except Exception:
                pass
        self.left_frame.after(500, self._cycle_today_color)

    def create_calendar_section(self):
        # Title
        tk.Label(self.left_frame, text="CALENDAR", font=("bold", 30, "bold"), fg="#00FFFF", bg=BG).pack(pady=(0, 6))

        # Header with previous/next buttons
        header = tk.Frame(self.left_frame, bg=BG)
        header.pack(fill=tk.X, padx=2)
        tk.Button(header, text="◀", command=self.prev_month, bg=BG, fg="#00FFFF", relief=tk.FLAT).pack(side=tk.LEFT)
        self.month_label = tk.Label(header, text="", font=("bold", 25, "bold"), fg="#FFFF00", bg=BG)
        self.month_label.pack(side=tk.LEFT, expand=True)
        tk.Button(header, text="▶", command=self.next_month, bg=BG, fg="#00FFFF", relief=tk.FLAT).pack(side=tk.LEFT)
        self.update_header()

        # Calendar grid area
        self.create_grid()

    def create_grid(self):
        # Remove old grid if exists
        if hasattr(self, 'grid_frame'):
            self.grid_frame.destroy()

        self.grid_frame = tk.Frame(self.left_frame, bg=BG)
        self.grid_frame.pack(pady=(7, 0))

        # Weekday headers (Sun..Sat)
        weekdays = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat" , "Sun"]
        for i, day in enumerate(weekdays):
            tk.Label(self.grid_frame, text=day, font=("bold", 20, "bold"), fg="#01F425", bg=BG).grid(row=0, column=i, padx=2, pady=2)

        # Calendar data (monthcalendar returns weeks starting Mon by default,
        # but we set headers as Sun..Sat. Use calendar.monthcalendar with firstweekday=6 if needed.
        cal = calendar.monthcalendar(self.current_year, self.current_month)
        # monthcalendar by default starts weeks on Monday; convert to Sun-first rows for consistent display
        # Construct rows where columns 0..6 correspond to Sun..Sat:
        # We'll map index: (Mon=0..Sun=6) -> target col = (weekday + 1) % 7
        self.today_label = None

        for week_idx, week in enumerate(cal, start=1):
            for mon_idx, day_num in enumerate(week):
                col = (mon_idx + 0) % 7  # remap Mon0->col0 ... Sun6->col6
                if day_num == 0:
                    lbl = tk.Label(self.grid_frame, text="", font=("bold", 10), bg=BG, width=3)
                else:
                    # Highlight ONLY if viewing Dec 2025 and this is the real today
                    is_today = (
                        self.current_month == self.real_today.month and
                        self.current_year == self.real_today.year and
                        day_num == self.real_today.day
                    )
                    fg = "#39FF14" if is_today else "#00FF00"
                    lbl = tk.Label(self.grid_frame, text=str(day_num), font=("bold", 11), fg=fg, bg=BG, width=3)
                    if is_today:
                        self.today_label = lbl
                lbl.grid(row=week_idx, column=col, padx=1, pady=1)

    def create_reminder_section(self):
        tk.Label(self.left_frame, text="EVENTS DEC 2025", font=("bold", 12, "bold"), fg="#FFFF00", bg=BG).pack(anchor="w", pady=(10, 4))

        self.reminder_text = tk.Text(self.left_frame, height=8, width=34, bg="#111", fg="#FF00FF", font=("bold", 10), state='disabled', wrap=tk.WORD)
        self.reminder_text.pack(fill=tk.X, padx=2)

        btn_frame = tk.Frame(self.left_frame, bg=BG)
        btn_frame.pack(fill=tk.X, pady=8)
        tk.Button(btn_frame, text="Add Event", command=self.add_reminder, bg="#222", fg="#39FF14", font=("bold", 10)).pack(side=tk.LEFT, padx=2)
        tk.Button(btn_frame, text="Refresh", command=self.refresh_display, bg="#222", fg="#FFFF00", font=("bold", 10)).pack(side=tk.LEFT, padx=2)
        tk.Button(btn_frame, text="Save", command=self.save_reminders, bg="#222", fg="#FF073A", font=("bold", 10)).pack(side=tk.LEFT, padx=2)

        self.update_reminder_display()

    def update_header(self):
        self.month_label.config(text=f"{calendar.month_name[self.current_month]} {self.current_year}")

    def update_reminder_display(self):
        lines = []
        for day in sorted(REMINDERS):
            for event in REMINDERS[day]:
                lines.append(f"• Dec {day}: {event}")
        if not lines:
            lines = ["No events scheduled."]

        self.reminder_text.config(state='normal')
        self.reminder_text.delete(1.0, tk.END)
        self.reminder_text.insert(tk.END, "\n".join(lines))
        self.reminder_text.config(state='disabled')

    def add_reminder(self):
        top = tk.Toplevel(self.parent, bg=BG)
        top.title("Add Reminder")
        top.geometry("320x180")
        top.transient(self.parent)
        top.grab_set()

        tk.Label(top, text="Day (1-31):", fg="#00FFFF", bg=BG).pack(pady=(8, 0))
        day_entry = tk.Entry(top, width=10, font=("bold", 12))
        day_entry.pack()

        tk.Label(top, text="Event:", fg="#FFFF00", bg=BG).pack(pady=(8, 0))
        event_entry = tk.Entry(top, width=30, font=("bold", 12))
        event_entry.pack()

        def submit():
            try:
                day = int(day_entry.get())
                event = event_entry.get().strip()
                if 1 <= day <= 31 and event:
                    REMINDERS.setdefault(day, []).append(event)
                    self.update_reminder_display()
                    top.destroy()
                else:
                    messagebox.showerror("Invalid", "Invalid day or empty event")
            except ValueError:
                messagebox.showerror("Invalid", "Day must be a number!")

        tk.Button(top, text="Add", command=submit, bg="#222", fg="#00FFFF", font=("bold", 10)).pack(pady=10)

    def refresh_display(self):
        # redraw calendar and reminders
        self.update_header()
        self.create_grid()
        self.update_reminder_display()

    def save_reminders(self):
        # Placeholder: show info (could write to file/JSON)
        messagebox.showinfo("Save", "Reminders saved (placeholder).")

    def prev_month(self):
        if self.current_month == 1:
            self.current_month = 12
            self.current_year -= 1
        else:
            self.current_month -= 1
        self.refresh_display()

    def next_month(self):
        if self.current_month == 12:
            self.current_month = 1
            self.current_year += 1
        else:
            self.current_month += 1
        self.refresh_display()

# Instantiate the calendar widget (defaults to Dec 2025)
calendar_widget = NeonCalendarWidget(root, month=12, year=2025)

# Bottom-left buttons (Save, Refresh, Reset, Exit)
bottom_left_frame = tk.Frame(calendar_widget.left_frame, bg=BG)
bottom_left_frame.pack(side=tk.BOTTOM, fill=tk.X, pady=6)

def save():
    print("Save Clicked")
    calendar_widget.save_reminders()

def refresh():
    print("Refresh Clicked")
    calendar_widget.refresh_display()

def reset():
    print("Reset Clicked")
    global REMINDERS
    REMINDERS = {5: ["Doctor Appointment at 2 pm"], 12: ["Project Meeting", "Submit Report"], 25: ["Christmas Day 🎄"]}
    calendar_widget.update_reminder_display()

def exit_app():
    root.destroy()

for text, cmd in [("Save", save), ("Refresh", refresh), ("Reset", reset), ("Exit", exit_app)]:
    tk.Button(bottom_left_frame, text=text, command=cmd, bg="#222", fg="#FF0019", font=("bold", 25)).pack(fill="x", pady=3)

# ---------------- TOP-CENTER FRAME ----------------
top_center = tk.Frame(root, bg=BG)
top_center.pack(side=tk.TOP, fill=tk.X, padx=6, pady=6)

title_label = tk.Label(top_center, text="Swiss", font=("bold", 32, "bold"), fg="gold", bg=BG)
title_label.pack(pady=(2, 4))

clock_label = tk.Label(top_center, font=("bold", 22), fg="lime", bg=BG)
clock_label.pack()

def update_clock():
    now = datetime.now()
    clock_label.config(text=now.strftime("%A %I:%M:%S%p"), fg=NEON_COLORS[now.second % len(NEON_COLORS)])
    root.after(1000, update_clock)

update_clock()

# Marquee
marquee_text = "  Welcome to Swiss Python Automate Dashboard!  "
marquee_label = tk.Label(top_center, text=marquee_text, font=("bold", 14), fg="cyan", bg=BG)
marquee_label.pack(pady=(4,6))

def scroll_marquee():
    global marquee_text
    marquee_text = marquee_text[1:] + marquee_text[0]
    marquee_label.config(text=marquee_text, fg=random.choice(NEON_COLORS))
    root.after(200, scroll_marquee)

scroll_marquee()

# Gold Skull Image
if PIL_AVAILABLE:
    try:
        skull_img = Image.open(BG_IMAGE_PATH)
        skull_img = skull_img.resize((200, 200), Image.LANCZOS)
        skull_photo = ImageTk.PhotoImage(skull_img)
        skull_label = tk.Label(top_center, image=skull_photo, bg=BG)
        skull_label.image = skull_photo
        skull_label.pack(pady=10)
    except Exception as e:
        print(f"Image load failed: {e}")
        skull_label = tk.Label(top_center, text="[Gold Skull Image]", fg="yellow", bg=BG)
        skull_label.pack(pady=6)
else:
    skull_label = tk.Label(top_center, text="[Gold Skull Image]", fg="yellow", bg=BG)
    skull_label.pack(pady=6)

# ---------------- MIDDLE-CENTER FRAME (Search) ----------------
middle_center = tk.Frame(root, bg=BG)
middle_center.pack(side=tk.TOP, fill=tk.X, pady=6, padx=6)

search_var = tk.StringVar()
search_entry = tk.Entry(middle_center, textvariable=search_var, font=("bold", 20), bg="#050505", fg="lime", insertbackground="lime")
search_entry.pack(padx=10, fill="both", expand=True)
search_entry.insert(5, "Swiss Search")
search_entry.focus_set()

def run_search(event=None):
    query = search_var.get().strip()
    if not query:
        return
    if query.startswith("http://") or query.startswith("https://"):
        webbrowser.open(query)
    else:
        qs = urllib.parse.quote_plus(query)
        webbrowser.open(f"https://www.google.com/search?q={qs}")

search_entry.bind("<Return>", run_search)

# ---------------- BOTTOM-CENTER FRAME (Terminal/Matrix) ----------------
bottom_center = tk.Text(root, height=12, bg="black", fg="lime", font=("bold", 15))
bottom_center.pack(side=tk.BOTTOM, fill="both", expand=False, padx=6, pady=6)

# Pre-create a small pool of tags to reuse (avoids unlimited tag growth)
TAG_POOL_SIZE = 50
for i in range(TAG_POOL_SIZE):
    bottom_center.tag_config(f"mtag{i}", foreground=random.choice(NEON_COLORS))

matrix_index = 0

def update_terminal():
    global matrix_index
    char = random.choice(MATRIX_CHARS)
    tag_name = f"mtag{matrix_index % TAG_POOL_SIZE}"
    bottom_center.insert(tk.END, char, tag_name)
    bottom_center.see(tk.END)
    matrix_index += 1
    # Trim content to avoid huge memory use
    max_chars = 40000
    # simple trim by deleting first 200 chars when exceeding threshold
    if int(bottom_center.index('end-1c').replace('.', '')) > max_chars:
        bottom_center.delete("1.0", "2.0")
    root.after(SPEED, update_terminal)

update_terminal()

# ---------------- TOP-RIGHT FRAME ----------------
fullscreen_btn = tk.Button(root, text="Fullscreen", command=toggle_fullscreen, bg=BG, fg="red", font=("bold", 20))
fullscreen_btn.pack(side=tk.TOP, anchor="ne", padx=8, pady=6)

# ---------------- MIDDLE-RIGHT FRAME (Web + Mac Apps) ----------------
right_frame = tk.Frame(root, bg=BG)
right_frame.pack(side=tk.RIGHT, fill=tk.Y, padx=10, pady=10)

web_apps = {"Google Docs": "https://docs.google.com/", "ChatGPT": "https://chat.openai.com/", "YouTube": "https://www.youtube.com/", "D2L": "http://d2l.com/"}
tk.Label(right_frame, text="URL", font=("blod", 25, "bold"), fg="#FF0303", bg=BG).pack(pady=5)
for name, url in web_apps.items():
    tk.Button(right_frame, text=name, command=lambda u=url: webbrowser.open(u), bg="#222", fg="#FF0000", font=("bold", 12)).pack(fill="y", pady=3)

mac_apps = {"Terminal": "Terminal", "Documents": "Documents"}
tk.Label(right_frame, text="Apps", font=("bold", 20 , "bold"), fg="#FF0000", bg=BG).pack(pady=8)
for name, app in mac_apps.items():
    tk.Button(right_frame, text=name, command=lambda a=app: os.system(f"open -a {a}"), bg="#222", fg="#FF0000", font=("bold", 12)).pack(fill="both", pady=3)

def add_web_app():
    name = simpledialog.askstring("Add Web App", "Name:")
    url = simpledialog.askstring("Add Web App", "URL:")
    if name and url:
        web_apps[name] = url
        messagebox.showinfo("Added", f"Added web app: {name}")

def add_mac_app():
    name = simpledialog.askstring("Add Mac App", "Name:")
    app = simpledialog.askstring("Add Mac App", "App Name:")
    if name and app:
        mac_apps[name] = app
        messagebox.showinfo("Added", f"Added mac app: {name}")

tk.Button(right_frame, text="Add Web App", command=add_web_app, bg="#222", fg="#FF0000", font=("bold", 10)).pack(fill="x", pady=4)
tk.Button(right_frame, text="Add Mac App", command=add_mac_app, bg="#222", fg="#FF0000", font=("bold", 10)).pack(fill="x", pady=4)

# ---------------- START the app ----------------
root.mainloop()


In [4]:
import tkinter as tk
from tkinter import simpledialog, messagebox
from datetime import datetime
import calendar
import webbrowser
import os
import random
import urllib.parse

try:
    from PIL import Image, ImageTk
    PIL_AVAILABLE = True
except Exception:
    PIL_AVAILABLE = False

# ---------------- SETTINGS ----------------
NEON_COLORS = ["#39FF14", "#FF073A", "#00F5FF", "#FFFF00", "#DA00FF", "#00FFFF"]
MATRIX_CHARS = "0123456789ABCDEFGJHIKOMPZ"
SPEED = 25  # Terminal speed in ms (lower -> faster)
REMINDERS = {5: ["Doctor Appointment at 2 pm"], 12: ["Project Meeting", "Submit Report"], 25: ["Christmas Day 🎄"]}
BG_IMAGE_PATH = "/Users/FreddyDirectory/Picture/GS.png"  # Path to gold skull image on your system

BG = "#000000"


# ---------------- MAIN WINDOW ----------------
root = tk.Tk()
root.title("Swiss Python Automate Dashboard")
root.geometry("1200x700")
root.configure(bg=BG)
fullscreen = False

def toggle_fullscreen():
    global fullscreen
    fullscreen = not fullscreen
    root.attributes("-fullscreen", fullscreen)

# ---------------- LEFT FRAME (Calendar + Reminders) ----------------
class NeonCalendarWidget:
    def __init__(self, parent, month=None, year=None):
        self.parent = parent
        self.real_today = datetime.today().date()
        today = datetime.today()
        self.current_month = month if month is not None else today.month
        self.current_year = year if year is not None else today.year

        # Main left container (wider)
        self.left_frame = tk.Frame(parent, bg=BG, width=380)
        self.left_frame.pack(side=tk.LEFT, fill=tk.Y, padx=8, pady=8)
        self.left_frame.pack_propagate(False)

        # UI elements placeholders
        self.today_label = None

        # Build UI
        self.create_calendar_section()
        self.create_reminder_section()

        # Start color cycle for today's label
        self._cycle_today_color()

    def _cycle_today_color(self):
        # Animate today's label with neon colors if it exists
        if self.today_label is not None:
            color = random.choice(NEON_COLORS)
            try:
                self.today_label.config(fg=color)
            except Exception:
                pass
        self.left_frame.after(500, self._cycle_today_color)

    def create_calendar_section(self):
        # Title
        tk.Label(self.left_frame, text="CALENDAR", font=("Helvetica", 26, "bold"), fg="#00FFFF", bg=BG).pack(pady=(2, 6))

        # Header with previous/next buttons
        header = tk.Frame(self.left_frame, bg=BG)
        header.pack(fill=tk.X, padx=4)
        tk.Button(header, text="◀", command=self.prev_month, bg=BG, fg="#00FFFF", relief=tk.FLAT, font=("Helvetica", 12, "bold")).pack(side=tk.LEFT)
        self.month_label = tk.Label(header, text="", font=("Helvetica", 18, "bold"), fg="#FFFF00", bg=BG)
        self.month_label.pack(side=tk.LEFT, expand=True)
        tk.Button(header, text="▶", command=self.next_month, bg=BG, fg="#00FFFF", relief=tk.FLAT, font=("Helvetica", 12, "bold")).pack(side=tk.LEFT)
        self.update_header()

        # Calendar grid area
        self.create_grid()

    def create_grid(self):
        # Remove old grid if exists
        if hasattr(self, 'grid_frame'):
            self.grid_frame.destroy()

        self.grid_frame = tk.Frame(self.left_frame, bg=BG)
        self.grid_frame.pack(pady=(8, 4))

        # Weekday headers (Sun..Sat)
        weekdays = ["Sun", "Mon", "Tue", "Wed", "Thu", "Fri", "Sat"]
        for i, day in enumerate(weekdays):
            tk.Label(self.grid_frame, text=day, font=("Helvetica", 12, "bold"), fg="#01F425", bg=BG, width=5).grid(row=0, column=i, padx=2, pady=2)

        # Use calendar.Calendar with Sunday as first weekday so rows match headers
        cal = calendar.Calendar(firstweekday=6).monthdayscalendar(self.current_year, self.current_month)

        self.today_label = None

        for week_idx, week in enumerate(cal, start=1):
            for col, day_num in enumerate(week):
                if day_num == 0:
                    lbl = tk.Label(self.grid_frame, text="", font=("Helvetica", 11), bg=BG, width=5, height=2, relief=tk.FLAT)
                else:
                    # Highlight if this matches the real today
                    is_today = (
                        self.current_month == self.real_today.month and
                        self.current_year == self.real_today.year and
                        day_num == self.real_today.day
                    )
                    fg = "#39FF14" if is_today else "#00FF00"
                    lbl = tk.Label(self.grid_frame, text=str(day_num), font=("Helvetica", 12, "bold"),
                                   fg=fg, bg=BG, width=5, height=2, relief=tk.FLAT)
                    if is_today:
                        self.today_label = lbl
                    # Bind click to show events for that day
                    lbl.bind("<Button-1>", lambda e, d=day_num: self.show_day_events(d))
                lbl.grid(row=week_idx, column=col, padx=1, pady=1, sticky="nsew")

        # Make columns expand evenly
        for c in range(7):
            self.grid_frame.grid_columnconfigure(c, weight=1)

    def create_reminder_section(self):
        tk.Label(self.left_frame, text="EVENTS", font=("Helvetica", 12, "bold"), fg="#FFFF00", bg=BG).pack(anchor="w", pady=(8, 4))

        self.reminder_text = tk.Text(self.left_frame, height=10, width=44, bg="#111", fg="#FF00FF", font=("Helvetica", 11), state='disabled', wrap=tk.WORD)
        self.reminder_text.pack(fill=tk.X, padx=4)

        btn_frame = tk.Frame(self.left_frame, bg=BG)
        btn_frame.pack(fill=tk.X, pady=8)
        tk.Button(btn_frame, text="Add Event", command=self.add_reminder, bg="#222", fg="#39FF14", font=("Helvetica", 11, "bold")).pack(side=tk.LEFT, padx=3)
        tk.Button(btn_frame, text="Refresh", command=self.refresh_display, bg="#222", fg="#FFFF00", font=("Helvetica", 11, "bold")).pack(side=tk.LEFT, padx=3)
        tk.Button(btn_frame, text="Save", command=self.save_reminders, bg="#222", fg="#FF073A", font=("Helvetica", 11, "bold")).pack(side=tk.LEFT, padx=3)

        self.update_reminder_display()

    def update_header(self):
        self.month_label.config(text=f"{calendar.month_name[self.current_month]} {self.current_year}")

    def update_reminder_display(self):
        lines = []
        # show only events for the currently visible month (assumes stored as day keys)
        for day in sorted(REMINDERS):
            for event in REMINDERS[day]:
                lines.append(f"• {calendar.month_name[self.current_month]} {day}: {event}")
        if not lines:
            lines = ["No events scheduled."]

        self.reminder_text.config(state='normal')
        self.reminder_text.delete(1.0, tk.END)
        self.reminder_text.insert(tk.END, "\n".join(lines))
        self.reminder_text.config(state='disabled')

    def add_reminder(self):
        top = tk.Toplevel(self.parent, bg=BG)
        top.title("Add Reminder")
        top.geometry("360x200")
        top.transient(self.parent)
        top.grab_set()

        tk.Label(top, text="Day (1-31):", fg="#00FFFF", bg=BG).pack(pady=(8, 0))
        day_entry = tk.Entry(top, width=10, font=("Helvetica", 12))
        day_entry.pack()

        tk.Label(top, text="Event:", fg="#FFFF00", bg=BG).pack(pady=(8, 0))
        event_entry = tk.Entry(top, width=36, font=("Helvetica", 12))
        event_entry.pack()

        def submit():
            try:
                day = int(day_entry.get())
                event = event_entry.get().strip()
                if 1 <= day <= 31 and event:
                    REMINDERS.setdefault(day, []).append(event)
                    self.update_reminder_display()
                    top.destroy()
                else:
                    messagebox.showerror("Invalid", "Invalid day or empty event")
            except ValueError:
                messagebox.showerror("Invalid", "Day must be a number!")

        tk.Button(top, text="Add", command=submit, bg="#222", fg="#00FFFF", font=("Helvetica", 12, "bold")).pack(pady=12)

    def show_day_events(self, day):
        # Popup showing events for a day
        events = REMINDERS.get(day, [])
        if events:
            messagebox.showinfo(f"Events - {day}", "\n".join(f"- {e}" for e in events))
        else:
            messagebox.showinfo(f"Events - {day}", "No events for this day.")

    def refresh_display(self):
        # redraw calendar and reminders
        self.update_header()
        self.create_grid()
        self.update_reminder_display()

    def save_reminders(self):
        # Placeholder: show info (could write to file/JSON)
        messagebox.showinfo("Save", "Reminders saved (placeholder).")

    def prev_month(self):
        if self.current_month == 1:
            self.current_month = 12
            self.current_year -= 1
        else:
            self.current_month -= 1
        self.refresh_display()

    def next_month(self):
        if self.current_month == 12:
            self.current_month = 1
            self.current_year += 1
        else:
            self.current_month += 1
        self.refresh_display()

# Instantiate the calendar widget (defaults to current month/year)
calendar_widget = NeonCalendarWidget(root, month=12, year=2025)

# Bottom-left buttons (Save, Refresh, Reset, Exit)
bottom_left_frame = tk.Frame(calendar_widget.left_frame, bg=BG)
bottom_left_frame.pack(side=tk.BOTTOM, fill=tk.X, pady=6)

def save():
    calendar_widget.save_reminders()

def refresh():
    calendar_widget.refresh_display()

def reset():
    global REMINDERS
    REMINDERS = {5: ["Doctor Appointment at 2 pm"], 12: ["Project Meeting", "Submit Report"], 25: ["Christmas Day 🎄"]}
    calendar_widget.update_reminder_display()

def exit_app():
    root.destroy()

for text, cmd in [("Save", save), ("Refresh", refresh), ("Reset", reset), ("Exit", exit_app)]:
    tk.Button(bottom_left_frame, text=text, command=cmd, bg="#222", fg="#FF0019", font=("Helvetica", 14, "bold")).pack(fill="x", pady=4)

# ---------------- TOP-CENTER FRAME ----------------
top_center = tk.Frame(root, bg=BG)
top_center.pack(side=tk.TOP, fill=tk.X, padx=6, pady=6)

title_label = tk.Label(top_center, text="Swiss", font=("Helvetica", 32, "bold"), fg="gold", bg=BG)
title_label.pack(pady=(2, 4))

clock_label = tk.Label(top_center, font=("Helvetica", 20), fg="lime", bg=BG)
clock_label.pack()

def update_clock():
    now = datetime.now()
    clock_label.config(text=now.strftime("%A %I:%M:%S %p"), fg=NEON_COLORS[now.second % len(NEON_COLORS)])
    root.after(1000, update_clock)

update_clock()

# Marquee
marquee_text = "  Welcome to Swiss Python Automate Dashboard!  "
marquee_label = tk.Label(top_center, text=marquee_text, font=("Helvetica", 14, "bold"), fg="cyan", bg=BG)
marquee_label.pack(pady=(4, 6))

def scroll_marquee():
    global marquee_text
    marquee_text = marquee_text[1:] + marquee_text[0]
    marquee_label.config(text=marquee_text, fg=random.choice(NEON_COLORS))
    root.after(200, scroll_marquee)

scroll_marquee()

# Gold Skull Image
if PIL_AVAILABLE:
    try:
        skull_img = Image.open(BG_IMAGE_PATH)
        skull_img = skull_img.resize((180, 180), Image.LANCZOS)
        skull_photo = ImageTk.PhotoImage(skull_img)
        skull_label = tk.Label(top_center, image=skull_photo, bg=BG)
        skull_label.image = skull_photo
        skull_label.pack(pady=8)
    except Exception as e:
        print(f"Image load failed: {e}")
        skull_label = tk.Label(top_center, text="[Gold Skull Image]", fg="yellow", bg=BG)
        skull_label.pack(pady=6)
else:
    skull_label = tk.Label(top_center, text="[Gold Skull Image]", fg="yellow", bg=BG)
    skull_label.pack(pady=6)

# ---------------- MIDDLE-CENTER FRAME (Search) ----------------
middle_center = tk.Frame(root, bg=BG)
middle_center.pack(side=tk.TOP, fill=tk.X, pady=6, padx=6)

search_var = tk.StringVar()
search_entry = tk.Entry(middle_center, textvariable=search_var, font=("Helvetica", 16), bg="#050505", fg="lime", insertbackground="lime")
search_entry.pack(padx=10, fill="both", expand=True)
search_entry.insert(0, "Swiss Search")
search_entry.focus_set()

def run_search(event=None):
    query = search_var.get().strip()
    if not query:
        return
    if query.startswith("http://") or query.startswith("https://"):
        webbrowser.open(query)
    else:
        qs = urllib.parse.quote_plus(query)
        webbrowser.open(f"https://www.google.com/search?q={qs}")

search_entry.bind("<Return>", run_search)

# ---------------- BOTTOM-CENTER FRAME (Terminal/Matrix) ----------------
bottom_center = tk.Text(root, height=12, bg="black", fg="lime", font=("Courier", 14))
bottom_center.pack(side=tk.BOTTOM, fill="both", expand=False, padx=6, pady=6)

# Pre-create a small pool of tags to reuse (avoids unlimited tag growth)
TAG_POOL_SIZE = 60
for i in range(TAG_POOL_SIZE):
    bottom_center.tag_config(f"mtag{i}", foreground=random.choice(NEON_COLORS))

matrix_index = 0

def update_terminal():
    global matrix_index
    char = random.choice(MATRIX_CHARS)
    tag_name = f"mtag{matrix_index % TAG_POOL_SIZE}"
    bottom_center.insert(tk.END, char, tag_name)
    bottom_center.see(tk.END)
    matrix_index += 1
    # Trim content to avoid huge memory use
    max_chars = 45000
    current_len = len(bottom_center.get("1.0", "end-1c"))
    if current_len > max_chars:
        # remove the first chunk of characters (keep it simple)
        bottom_center.delete("1.0", "1.2000")  # deletes roughly first 2000 chars (approx)
    root.after(SPEED, update_terminal)

update_terminal()

# ---------------- TOP-RIGHT FRAME ----------------
fullscreen_btn = tk.Button(root, text="Fullscreen", command=toggle_fullscreen, bg=BG, fg="red", font=("Helvetica", 12, "bold"))
fullscreen_btn.pack(side=tk.TOP, anchor="ne", padx=8, pady=6)

# ---------------- MIDDLE-RIGHT FRAME (Web + Mac Apps) ----------------
right_frame = tk.Frame(root, bg=BG)
right_frame.pack(side=tk.RIGHT, fill=tk.Y, padx=10, pady=10)

web_apps = {"Google Docs": "https://docs.google.com/", "ChatGPT": "https://chat.openai.com/", "YouTube": "https://www.youtube.com/", "D2L": "http://d2l.com/"}
tk.Label(right_frame, text="URL", font=("Helvetica", 18, "bold"), fg="#FF0303", bg=BG).pack(pady=5)
for name, url in web_apps.items():
    tk.Button(right_frame, text=name, command=lambda u=url: webbrowser.open(u), bg="#222", fg="#FF0000", font=("Helvetica", 11)).pack(fill="y", pady=3)

mac_apps = {"Terminal": "Terminal", "Documents": "Documents"}
tk.Label(right_frame, text="Apps", font=("Helvetica", 16, "bold"), fg="#FF0000", bg=BG).pack(pady=8)
for name, app in mac_apps.items():
    tk.Button(right_frame, text=name, command=lambda a=app: os.system(f"open -a {a}"), bg="#222", fg="#FF0000", font=("Helvetica", 11)).pack(fill="both", pady=3)

def add_web_app():
    name = simpledialog.askstring("Add Web App", "Name:")
    url = simpledialog.askstring("Add Web App", "URL:")
    if name and url:
        web_apps[name] = url
        messagebox.showinfo("Added", f"Added web app: {name}")

def add_mac_app():
    name = simpledialog.askstring("Add Mac App", "Name:")
    app = simpledialog.askstring("Add Mac App", "App Name:")
    if name and app:
        mac_apps[name] = app
        messagebox.showinfo("Added", f"Added mac app: {name}")

tk.Button(right_frame, text="Add Web App", command=add_web_app, bg="#222", fg="#FF0000", font=("Helvetica", 10)).pack(fill="x", pady=4)
tk.Button(right_frame, text="Add Mac App", command=add_mac_app, bg="#222", fg="#FF0000", font=("Helvetica", 10)).pack(fill="x", pady=4)

# ---------------- START the app ----------------
root.mainloop()


In [6]:
import tkinter as tk
from tkinter import Y, simpledialog, messagebox
from datetime import datetime
import calendar
import webbrowser
import os
import random
import urllib.parse 

try:
    from PIL import Image, ImageTk
    PIL_AVAILABLE = True
except Exception:
    PIL_AVAILABLE = False

# ---------------- SETTINGS ----------------
NEON_COLORS = ["#39FF14", "#FF073A", "#00F5FF", "#FFFF00", "#DA00FF", "#00FFFF"]
MATRIX_CHARS = "0123456789ABCDEFGJHIKOMPZ"
SPEED = 1 # Terminal speed in ms (lower -> faster)
REMINDERS = {5: ["Doctor Appointment at 2 pm"], 12: ["Project Meeting", "Submit Report"], 25: ["Christmas Day 🎄"]}
BG_IMAGE_PATH = "/Users/FreddyDirectory/Picture/GS.PNG" # Path to gold skull image on your system   
BG = "#000000"

# ---------------- MAIN WINDOW ----------------
root = tk.Tk()
root.title("Swiss Python Automate Dashboard")
root.geometry("1200x700")
root.configure(bg=BG)
fullscreen = False

def toggle_fullscreen():
    global fullscreen
    fullscreen = not fullscreen
    root.attributes("-fullscreen", fullscreen)

# ---------------- LEFT FRAME (Calendar + Reminders) ----------------
class NeonCalendarWidget:
    def __init__(self, parent, month=12, year=2025):
        self.parent = parent
        self.real_today = datetime.today().date()
        self.current_month = month
        self.current_year = year

        # Main left container
        self.left_frame = tk.Frame(parent, bg=BG, width=320)
        self.left_frame.pack(side=tk.LEFT, fill=tk.Y, padx=6, pady=6)

        # UI elements placeholders
        self.today_label = None

        # Build UI
        self.create_calendar_section()
        self.create_reminder_section()

        # Start color cycle for today's label
        self._cycle_today_color()

    def _cycle_today_color(self):
        # Animate today's label with neon colors if it exists
        if self.today_label is not None:
            color = random.choice(NEON_COLORS)
            try:
                self.today_label.config(fg=color)
            except Exception:
                pass
        self.left_frame.after(500, self._cycle_today_color)

    def create_calendar_section(self):
        # Title
        tk.Label(self.left_frame, text="CALENDAR", font=("bold", 30, "bold"), fg="#00FFFF", bg=BG).pack(pady=(0, 6))

        # Header with previous/next buttons
        header = tk.Frame(self.left_frame, bg=BG)
        header.pack(fill=tk.X, padx=2)
        tk.Button(header, text="◀", command=self.prev_month, bg=BG, fg="#00FFFF", relief=tk.FLAT).pack(side=tk.LEFT)
        self.month_label = tk.Label(header, text="", font=("bold", 25, "bold"), fg="#FFFF00", bg=BG)
        self.month_label.pack(side=tk.LEFT, expand=True)
        tk.Button(header, text="▶", command=self.next_month, bg=BG, fg="#00FFFF", relief=tk.FLAT).pack(side=tk.LEFT)
        self.update_header()

        # Calendar grid area
        self.create_grid()

    def create_grid(self):
        # Remove old grid if exists
        if hasattr(self, 'grid_frame'):
            self.grid_frame.destroy()

        self.grid_frame = tk.Frame(self.left_frame, bg=BG)
        self.grid_frame.pack(pady=(7, 0))

        # Weekday headers (Sun..Sat)
        weekdays = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat" , "Sun"]
        for i, day in enumerate(weekdays):
            tk.Label(self.grid_frame, text=day, font=("bold", 20, "bold"), fg="#01F425", bg=BG).grid(row=0, column=i, padx=2, pady=2)

        # Calendar data (monthcalendar returns weeks starting Mon by default,
        # but we set headers as Sun..Sat. Use calendar.monthcalendar with firstweekday=6 if needed.
        cal = calendar.monthcalendar(self.current_year, self.current_month)
        # monthcalendar by default starts weeks on Monday; convert to Sun-first rows for consistent display
        # Construct rows where columns 0..6 correspond to Sun..Sat:
        # We'll map index: (Mon=0..Sun=6) -> target col = (weekday + 1) % 7
        self.today_label = None

        for week_idx, week in enumerate(cal, start=1):
            for mon_idx, day_num in enumerate(week):
                col = (mon_idx + 0) % 7  # remap Mon0->col0 ... Sun6->col6
                if day_num == 0:
                    lbl = tk.Label(self.grid_frame, text="", font=("bold", 10), bg=BG, width=3)
                else:
                    # Highlight ONLY if viewing Dec 2025 and this is the real today
                    is_today = (
                        self.current_month == self.real_today.month and
                        self.current_year == self.real_today.year and
                        day_num == self.real_today.day
                    )
                    fg = "#39FF14" if is_today else "#00FF00"
                    lbl = tk.Label(self.grid_frame, text=str(day_num), font=("bold", 11), fg=fg, bg=BG, width=3)
                    if is_today:
                        self.today_label = lbl
                lbl.grid(row=week_idx, column=col, padx=1, pady=1)

    def create_reminder_section(self):
        tk.Label(self.left_frame, text="EVENTS DEC 2025", font=("bold", 12, "bold"), fg="#FFFF00", bg=BG).pack(anchor="w", pady=(10, 4))

        self.reminder_text = tk.Text(self.left_frame, height=8, width=34, bg="#111", fg="#FF00FF", font=("bold", 10), state='disabled', wrap=tk.WORD)
        self.reminder_text.pack(fill=tk.X, padx=2)

        btn_frame = tk.Frame(self.left_frame, bg=BG)
        btn_frame.pack(fill=tk.X, pady=8)
        tk.Button(btn_frame, text="Add Event", command=self.add_reminder, bg="#222", fg="#39FF14", font=("bold", 10)).pack(side=tk.LEFT, padx=2)
        tk.Button(btn_frame, text="Refresh", command=self.refresh_display, bg="#222", fg="#FFFF00", font=("bold", 10)).pack(side=tk.LEFT, padx=2)
        tk.Button(btn_frame, text="Save", command=self.save_reminders, bg="#222", fg="#FF073A", font=("bold", 10)).pack(side=tk.LEFT, padx=2)

        self.update_reminder_display()

    def update_header(self):
        self.month_label.config(text=f"{calendar.month_name[self.current_month]} {self.current_year}")

    def update_reminder_display(self):
        lines = []
        for day in sorted(REMINDERS):
            for event in REMINDERS[day]:
                lines.append(f"• Dec {day}: {event}")
        if not lines:
            lines = ["No events scheduled."]

        self.reminder_text.config(state='normal')
        self.reminder_text.delete(1.0, tk.END)
        self.reminder_text.insert(tk.END, "\n".join(lines))
        self.reminder_text.config(state='disabled')

    def add_reminder(self):
        top = tk.Toplevel(self.parent, bg=BG)
        top.title("Add Reminder")
        top.geometry("320x180")
        top.transient(self.parent)
        top.grab_set()

        tk.Label(top, text="Day (1-31):", fg="#00FFFF", bg=BG).pack(pady=(8, 0))
        day_entry = tk.Entry(top, width=10, font=("bold", 12))
        day_entry.pack()

        tk.Label(top, text="Event:", fg="#FFFF00", bg=BG).pack(pady=(8, 0))
        event_entry = tk.Entry(top, width=30, font=("bold", 12))
        event_entry.pack()

        def submit():
            try:
                day = int(day_entry.get())
                event = event_entry.get().strip()
                if 1 <= day <= 31 and event:
                    REMINDERS.setdefault(day, []).append(event)
                    self.update_reminder_display()
                    top.destroy()
                else:
                    messagebox.showerror("Invalid", "Invalid day or empty event")
            except ValueError:
                messagebox.showerror("Invalid", "Day must be a number!")

        tk.Button(top, text="Add", command=submit, bg="#222", fg="#00FFFF", font=("bold", 10)).pack(pady=10)

    def refresh_display(self):
        # redraw calendar and reminders
        self.update_header()
        self.create_grid()
        self.update_reminder_display()

    def save_reminders(self):
        # Placeholder: show info (could write to file/JSON)
        messagebox.showinfo("Save", "Reminders saved (placeholder).")

    def prev_month(self):
        if self.current_month == 1:
            self.current_month = 12
            self.current_year -= 1
        else:
            self.current_month -= 1
        self.refresh_display()

    def next_month(self):
        if self.current_month == 12:
            self.current_month = 1
            self.current_year += 1
        else:
            self.current_month += 1
        self.refresh_display()

# Instantiate the calendar widget (defaults to Dec 2025)
calendar_widget = NeonCalendarWidget(root, month=12, year=2025)

# Bottom-left buttons (Save, Refresh, Reset, Exit)
bottom_left_frame = tk.Frame(calendar_widget.left_frame, bg=BG)
bottom_left_frame.pack(side=tk.BOTTOM, fill=tk.X, pady=6)

def save():
    print("Save Clicked")
    calendar_widget.save_reminders()

def refresh():
    print("Refresh Clicked")
    calendar_widget.refresh_display()

def reset():
    print("Reset Clicked")
    global REMINDERS
    REMINDERS = {5: ["Doctor Appointment at 2 pm"], 12: ["Project Meeting", "Submit Report"], 25: ["Christmas Day 🎄"]}
    calendar_widget.update_reminder_display()

def exit_app():
    root.destroy()

for text, cmd in [("Save", save), ("Refresh", refresh), ("Reset", reset), ("Exit", exit_app)]:
    tk.Button(bottom_left_frame, text=text, command=cmd, bg="#222", fg="#FF0019", font=("bold", 25)).pack(fill="x", pady=3)

# ---------------- TOP-CENTER FRAME ----------------
top_center = tk.Frame(root, bg=BG)
top_center.pack(side=tk.TOP, fill=tk.X, padx=6, pady=6)

title_label = tk.Label(top_center, text="Swiss", font=("bold", 32, "bold"), fg="gold", bg=BG)
title_label.pack(pady=(2, 4))

clock_label = tk.Label(top_center, font=("bold", 22), fg="lime", bg=BG)
clock_label.pack()

def update_clock():
    now = datetime.now()
    clock_label.config(text=now.strftime("%A %I:%M:%S%p"), fg=NEON_COLORS[now.second % len(NEON_COLORS)])
    root.after(1000, update_clock)

update_clock()

# Marquee
marquee_text = "  Welcome to Swiss Python Automate Dashboard!  "
marquee_label = tk.Label(top_center, text=marquee_text, font=("bold", 14), fg="cyan", bg=BG)
marquee_label.pack(pady=(4,6))

def scroll_marquee():
    global marquee_text
    marquee_text = marquee_text[1:] + marquee_text[0]
    marquee_label.config(text=marquee_text, fg=random.choice(NEON_COLORS))
    root.after(200, scroll_marquee)

scroll_marquee()

# Gold Skull Image
if PIL_AVAILABLE:
    try:
        skull_img = Image.open(BG_IMAGE_PATH)
        skull_img = skull_img.resize((200, 200), Image.LANCZOS)
        skull_photo = ImageTk.PhotoImage(skull_img)
        skull_label = tk.Label(top_center, image=skull_photo, bg=BG)
        skull_label.image = skull_photo
        skull_label.pack(pady=10)
    except Exception as e:
        print(f"Image load failed: {e}")
        skull_label = tk.Label(top_center, text="[Gold Skull Image]", fg="yellow", bg=BG)
        skull_label.pack(pady=6)
else:
    skull_label = tk.Label(top_center, text="[Gold Skull Image]", fg="yellow", bg=BG)
    skull_label.pack(pady=6)

# ---------------- MIDDLE-CENTER FRAME (Search) ----------------
middle_center = tk.Frame(root, bg=BG)
middle_center.pack(side=tk.TOP, fill=tk.X, pady=6, padx=6)

search_var = tk.StringVar()
search_entry = tk.Entry(middle_center, textvariable=search_var, font=("bold", 20), bg="#050505", fg="lime", insertbackground="lime")
search_entry.pack(padx=10, fill="both", expand=True)
search_entry.insert(5, "Swiss Search")
search_entry.focus_set()

def run_search(event=None):
    query = search_var.get().strip()
    if not query:
        return
    if query.startswith("http://") or query.startswith("https://"):
        webbrowser.open(query)
    else:
        qs = urllib.parse.quote_plus(query)
        webbrowser.open(f"https://www.google.com/search?q={qs}")

search_entry.bind("<Return>", run_search)

# ---------------- BOTTOM-CENTER FRAME (Terminal/Matrix) ----------------
bottom_center = tk.Text(root, height=12, bg="black", fg="lime", font=("bold", 15))
bottom_center.pack(side=tk.BOTTOM, fill="both", expand=False, padx=6, pady=6)

# Pre-create a small pool of tags to reuse (avoids unlimited tag growth)
TAG_POOL_SIZE = 50
for i in range(TAG_POOL_SIZE):
    bottom_center.tag_config(f"mtag{i}", foreground=random.choice(NEON_COLORS))

matrix_index = 0

def update_terminal():
    global matrix_index
    char = random.choice(MATRIX_CHARS)
    tag_name = f"mtag{matrix_index % TAG_POOL_SIZE}"
    bottom_center.insert(tk.END, char, tag_name)
    bottom_center.see(tk.END)
    matrix_index += 1
    # Trim content to avoid huge memory use
    max_chars = 40000
    # simple trim by deleting first 200 chars when exceeding threshold
    if int(bottom_center.index('end-1c').replace('.', '')) > max_chars:
        bottom_center.delete("1.0", "2.0")
    root.after(SPEED, update_terminal)

update_terminal()

# ---------------- TOP-RIGHT FRAME ----------------
fullscreen_btn = tk.Button(root, text="Fullscreen", command=toggle_fullscreen, bg=BG, fg="red", font=("bold", 20))
fullscreen_btn.pack(side=tk.TOP, anchor="ne", padx=8, pady=6)

# ---------------- MIDDLE-RIGHT FRAME (Web + Mac Apps) ----------------
right_frame = tk.Frame(root, bg=BG)
right_frame.pack(side=tk.RIGHT, fill=tk.Y, padx=10, pady=10)

web_apps = {"Google Docs": "https://docs.google.com/", "ChatGPT": "https://chat.openai.com/", "YouTube": "https://www.youtube.com/", "D2L": "http://d2l.com/"}
tk.Label(right_frame, text="URL", font=("blod", 25, "bold"), fg="#FF0303", bg=BG).pack(pady=5)
for name, url in web_apps.items():
    tk.Button(right_frame, text=name, command=lambda u=url: webbrowser.open(u), bg="#222", fg="#FF0000", font=("bold", 12)).pack(fill="y", pady=3)

mac_apps = {"Terminal": "Terminal", "Documents": "Documents"}
tk.Label(right_frame, text="Apps", font=("bold", 20 , "bold"), fg="#FF0000", bg=BG).pack(pady=8)
for name, app in mac_apps.items():
    tk.Button(right_frame, text=name, command=lambda a=app: os.system(f"open -a {a}"), bg="#222", fg="#FF0000", font=("bold", 12)).pack(fill="both", pady=3)

def add_web_app():
    name = simpledialog.askstring("Add Web App", "Name:")
    url = simpledialog.askstring("Add Web App", "URL:")
    if name and url:
        web_apps[name] = url
        messagebox.showinfo("Added", f"Added web app: {name}")

def add_mac_app():
    name = simpledialog.askstring("Add Mac App", "Name:")
    app = simpledialog.askstring("Add Mac App", "App Name:")
    if name and app:
        mac_apps[name] = app
        messagebox.showinfo("Added", f"Added mac app: {name}")

tk.Button(right_frame, text="Add Web App", command=add_web_app, bg="#222", fg="#FF0000", font=("bold", 10)).pack(fill="x", pady=4)
tk.Button(right_frame, text="Add Mac App", command=add_mac_app, bg="#222", fg="#FF0000", font=("bold", 10)).pack(fill="x", pady=4)

# ---------------- START the app ----------------
root.mainloop()


KeyboardInterrupt: 

In [1]:
from sys import _clear_internal_caches
import tkinter as tk
from tkinter import CENTER, Y, simpledialog, messagebox
from datetime import datetime
import calendar
import webbrowser
import os
import random
import urllib.parse 

try:
    from PIL import Image, ImageTk
    PIL_AVAILABLE = True
except Exception:
    PIL_AVAILABLE = False

# ---------------- SETTINGS ----------------
NEON_COLORS = ["#39FF14", "#FF073A", "#00F5FF", "#FFFF00", "#DA00FF", "#00FFFF"]
MATRIX_CHARS = "0123456789ABCDEFGJHIKOMPZ"
SPEED = 1 # Terminal speed in ms (lower -> faster)
REMINDERS = {5: ["Doctor Appointment at 2 pm"], 12: ["Project Meeting", "Submit Report"], 25: ["Christmas Day 🎄"]}
BG_IMAGE_PATH = "/Users/FreddyDirectory/Picture/GS.PNG" # Path to gold skull image on your system   
BG = "#000000"

# ---------------- MAIN WINDOW ----------------
root = tk.Tk()
root.title("Swiss Python Automate Dashboard")
root.geometry("1200x700")
root.configure(bg=BG)
fullscreen = False

def toggle_fullscreen():
    global fullscreen
    fullscreen = not fullscreen
    root.attributes("-fullscreen", fullscreen)

# ---------------- LEFT FRAME (Calendar + Reminders) ----------------
class NeonCalendarWidget:
    def __init__(self, parent, month=12, year=2025):
        self.parent = parent
        self.real_today = datetime.today().date()
        self.current_month = month
        self.current_year = year

        # Main left container
        self.left_frame = tk.Frame(parent, bg=BG, width=320)
        self.left_frame.pack(side=tk.LEFT, fill=tk.Y, padx=6, pady=6)

        # UI elements placeholders
        self.today_label = None

        # Build UI
        self.create_calendar_section()
        self.create_reminder_section()

        # Start color cycle for today's label
        self._cycle_today_color()

    def _cycle_today_color(self):
        # Animate today's label with neon colors if it exists
        if self.today_label is not None:
            color = random.choice(NEON_COLORS)
            try:
                self.today_label.config(fg=color)
            except Exception:
                pass
        self.left_frame.after(500, self._cycle_today_color)

    def create_calendar_section(self):
        # Title
        tk.Label(self.left_frame, text="CALENDAR", font=("bold", 30, "bold"), fg="#00FFFF", bg=BG).pack(pady=(0, 6))

        # Header with previous/next buttons
        header = tk.Frame(self.left_frame, bg=BG)
        header.pack(fill=tk.X, padx=2)
        tk.Button(header, text="◀", command=self.prev_month, bg=BG, fg="#00FFFF", relief=tk.FLAT).pack(side=tk.LEFT)
        self.month_label = tk.Label(header, text="", font=("bold", 25, "bold"), fg="#FFFF00", bg=BG)
        self.month_label.pack(side=tk.LEFT, expand=True)
        tk.Button(header, text="▶", command=self.next_month, bg=BG, fg="#00FFFF", relief=tk.FLAT).pack(side=tk.LEFT)
        self.update_header()

        # Calendar grid area
        self.create_grid()

    def create_grid(self):
        # Remove old grid if exists
        if hasattr(self, 'grid_frame'):
            self.grid_frame.destroy()

        self.grid_frame = tk.Frame(self.left_frame, bg=BG)
        self.grid_frame.pack(pady=(7, 0))

        # Weekday headers (Sun..Sat)
        weekdays = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat" , "Sun"]
        for i, day in enumerate(weekdays):
            tk.Label(self.grid_frame, text=day, font=("bold", 20, "bold"), fg="#01F425", bg=BG).grid(row=0, column=i, padx=2, pady=2)

        # Calendar data (monthcalendar returns weeks starting Mon by default,
        # but we set headers as Sun..Sat. Use calendar.monthcalendar with firstweekday=6 if needed.
        cal = calendar.monthcalendar(self.current_year, self.current_month)
        # monthcalendar by default starts weeks on Monday; convert to Sun-first rows for consistent display
        # Construct rows where columns 0..6 correspond to Sun..Sat:
        # We'll map index: (Mon=0..Sun=6) -> target col = (weekday + 1) % 7
        self.today_label = None

        for week_idx, week in enumerate(cal, start=1):
            for mon_idx, day_num in enumerate(week):
                col = (mon_idx + 0) % 7  # remap Mon0->col0 ... Sun6->col6
                if day_num == 0:
                    lbl = tk.Label(self.grid_frame, text="", font=("bold", 10), bg=BG, width=3)
                else:
                    # Highlight ONLY if viewing Dec 2025 and this is the real today
                    is_today = (
                        self.current_month == self.real_today.month and
                        self.current_year == self.real_today.year and
                        day_num == self.real_today.day
                    )
                    fg = "#39FF14" if is_today else "#00FF00"
                    lbl = tk.Label(self.grid_frame, text=str(day_num), font=("bold", 11), fg=fg, bg=BG, width=3)
                    if is_today:
                        self.today_label = lbl
                lbl.grid(row=week_idx, column=col, padx=1, pady=1)

    def create_reminder_section(self):
        tk.Label(self.left_frame, text="EVENTS DEC 2025", font=("bold", 12, "bold"), fg="#FFFF00", bg=BG).pack(anchor="w", pady=(10, 4))

        self.reminder_text = tk.Text(self.left_frame, height=8, width=34, bg="#111", fg="#FF00FF", font=("bold", 10), state='disabled', wrap=tk.WORD)
        self.reminder_text.pack(fill=tk.X, padx=2)

        btn_frame = tk.Frame(self.left_frame, bg=BG)
        btn_frame.pack(fill=tk.X, pady=8)
        tk.Button(btn_frame, text="Add Event", command=self.add_reminder, bg="#222", fg="#39FF14", font=("bold", 10)).pack(side=tk.LEFT, padx=2)
        tk.Button(btn_frame, text="Refresh", command=self.refresh_display, bg="#222", fg="#FFFF00", font=("bold", 10)).pack(side=tk.LEFT, padx=2)
        tk.Button(btn_frame, text="Save", command=self.save_reminders, bg="#222", fg="#FF073A", font=("bold", 10)).pack(side=tk.LEFT, padx=2)

        self.update_reminder_display()

    def update_header(self):
        self.month_label.config(text=f"{calendar.month_name[self.current_month]} {self.current_year}")

    def update_reminder_display(self):
        lines = []
        for day in sorted(REMINDERS):
            for event in REMINDERS[day]:
                lines.append(f"• Dec {day}: {event}")
        if not lines:
            lines = ["No events scheduled."]

        self.reminder_text.config(state='normal')
        self.reminder_text.delete(1.0, tk.END)
        self.reminder_text.insert(tk.END, "\n".join(lines))
        self.reminder_text.config(state='disabled')

    def add_reminder(self):
        top = tk.Toplevel(self.parent, bg=BG)
        top.title("Add Reminder")
        top.geometry("320x180")
        top.transient(self.parent)
        top.grab_set()

        tk.Label(top, text="Day (1-31):", fg="#00FFFF", bg=BG).pack(pady=(8, 0))
        day_entry = tk.Entry(top, width=10, font=("bold", 12))
        day_entry.pack()

        tk.Label(top, text="Event:", fg="#FFFF00", bg=BG).pack(pady=(8, 0))
        event_entry = tk.Entry(top, width=30, font=("bold", 12))
        event_entry.pack()

        def submit():
            try:
                day = int(day_entry.get())
                event = event_entry.get().strip()
                if 1 <= day <= 31 and event:
                    REMINDERS.setdefault(day, []).append(event)
                    self.update_reminder_display()
                    top.destroy()
                else:
                    messagebox.showerror("Invalid", "Invalid day or empty event")
            except ValueError:
                messagebox.showerror("Invalid", "Day must be a number!")

        tk.Button(top, text="Add", command=submit, bg="#222", fg="#00FFFF", font=("bold", 10)).pack(pady=10)

    def refresh_display(self):
        # redraw calendar and reminders
        self.update_header()
        self.create_grid()
        self.update_reminder_display()

    def save_reminders(self):
        # Placeholder: show info (could write to file/JSON)
        messagebox.showinfo("Save", "Reminders saved (placeholder).")

    def prev_month(self):
        if self.current_month == 1:
            self.current_month = 12
            self.current_year -= 1
        else:
            self.current_month -= 1
        self.refresh_display()

    def next_month(self):
        if self.current_month == 12:
            self.current_month = 1
            self.current_year += 1
        else:
            self.current_month += 1
        self.refresh_display()

# Instantiate the calendar widget (defaults to Dec 2025)
calendar_widget = NeonCalendarWidget(root, month=12, year=2025)

# Bottom-left buttons (Save, Refresh, Reset, Exit)
bottom_left_frame = tk.Frame(calendar_widget.left_frame, bg=BG)
bottom_left_frame.pack(side=tk.BOTTOM, fill=tk.X, pady=6)

def save():
    print("Save Clicked")
    calendar_widget.save_reminders()

def refresh():
    print("Refresh Clicked")
    calendar_widget.refresh_display()

def reset():
    print("Reset Clicked")
    global REMINDERS
    REMINDERS = {5: ["Doctor Appointment at 2 pm"], 12: ["Project Meeting", "Submit Report"], 25: ["Christmas Day 🎄"]}
    calendar_widget.update_reminder_display()

def exit_app():
    root.destroy()

for text, cmd in [("Save", save), ("Refresh", refresh), ("Reset", reset), ("Exit", exit_app)]:
    tk.Button(bottom_left_frame, text=text, command=cmd, bg="#222", fg="#FF0019", font=("bold", 25)).pack(fill="x", pady=3)

# ---------------- TOP-CENTER FRAME ----------------
top_center = tk.Frame(root, bg=BG)
top_center.pack(side=tk.TOP, fill=tk.X, padx=6, pady=6)

title_label = tk.Label(top_center, text="Swiss", font=("bold", 32, "bold"), fg="gold", bg=BG)
title_label.pack(pady=(2, 4))

clock_label = tk.Label(top_center, font=("bold", 22), fg="lime", bg=BG)
clock_label.pack()

def update_clock():
    now = datetime.now()
    clock_label.config(text=now.strftime("%A %I:%M:%S%p"), fg=NEON_COLORS[now.second % len(NEON_COLORS)])
    root.after(1000, update_clock)

update_clock()

# Marquee
marquee_text = "  Welcome to Swiss Python Automate Dashboard!  "
marquee_label = tk.Label(top_center, text=marquee_text, font=("bold", 14), fg="cyan", bg=BG)
marquee_label.pack(pady=(4,6))

def scroll_marquee():
    global marquee_text
    marquee_text = marquee_text[1:] + marquee_text[0]
    marquee_label.config(text=marquee_text, fg=random.choice(NEON_COLORS))
    root.after(200, scroll_marquee)

scroll_marquee()

# Gold Skull Image
if PIL_AVAILABLE:
    try:
        skull_img = Image.open(BG_IMAGE_PATH)
        skull_img = skull_img.resize((200, 200), Image.LANCZOS)
        skull_photo = ImageTk.PhotoImage(skull_img)
        skull_label = tk.Label(top_center, image=skull_photo, bg=BG)
        skull_label.image = skull_photo
        skull_label.pack(pady=10)
    except Exception as e:
        print(f"Image load failed: {e}")
        skull_label = tk.Label(top_center, text="[Gold Skull Image]", fg="yellow", bg=BG)
        skull_label.pack(pady=6)
else:
    skull_label = tk.Label(top_center, text="[Gold Skull Image]", fg="yellow", bg=BG)
    skull_label.pack(pady=6)

# ---------------- MIDDLE-CENTER FRAME (Search) ----------------
middle_center = tk.Frame(root, bg=BG)
middle_center.pack(side=tk.TOP, fill=tk.X, pady=6, padx=6)

search_var = tk.StringVar()
search_entry = tk.Entry(middle_center, textvariable=search_var, font=("bold", 20), bg="#050505", fg="lime", insertbackground="lime")
search_entry.pack(padx=10, fill="both", expand=True)
search_entry.insert(5, "Swiss Search")
search_entry.focus_set()

def run_search(event=None):
    query = search_var.get().strip()
    if not query:
        return
    if query.startswith("http://") or query.startswith("https://"):
        webbrowser.open(query)
    else:
        qs = urllib.parse.quote_plus(query)
        webbrowser.open(f"https://www.google.com/search?q={qs}")

search_entry.bind("<Return>", run_search)

# ---------------- BOTTOM-CENTER FRAME (Terminal/Matrix) ----------------
bottom_center = tk.Text(root, height=12, bg="black", fg="lime", font=("bold", 15))
bottom_center.pack(side=tk.BOTTOM, fill="both", expand=False, padx=6, pady=6)

# Pre-create a small pool of tags to reuse (avoids unlimited tag growth)
TAG_POOL_SIZE = 50
for i in range(TAG_POOL_SIZE):
    bottom_center.tag_config(f"mtag{i}", foreground=random.choice(NEON_COLORS))

matrix_index = 0

def update_terminal():
    global matrix_index
    char = random.choice(MATRIX_CHARS)
    tag_name = f"mtag{matrix_index % TAG_POOL_SIZE}"
    bottom_center.insert(tk.END, char, tag_name)
    bottom_center.see(tk.END)
    matrix_index += 1
    # Trim content to avoid huge memory use
    max_chars = 40000
    # simple trim by deleting first 200 chars when exceeding threshold
    if int(bottom_center.index('end-1c').replace('.', '')) > max_chars:
        bottom_center.delete("1.0", "2.0")
    root.after(SPEED, update_terminal)

update_terminal()

# ---------------- TOP-RIGHT FRAME ----------------
fullscreen_btn = tk.Button(root, text="Fullscreen", command=toggle_fullscreen, bg=BG, fg="red", font=("bold", 20))
fullscreen_btn.pack(side=tk.TOP, anchor="ne", padx=8, pady=6)

# ---------------- MIDDLE-RIGHT FRAME (Web + Mac Apps) ----------------
left = tk.Frame(root, bg=BG)
left.pack(side=tk.RIGHT, fill=tk.Y, padx=10, pady=10)

web_apps = {
    "Google Docs": "https://docs.google.com/", 
    "ChatGPT": "https://chat.openai.com/", 
    "YouTube": "https://www.youtube.com/", 
    "D2L": "http://d2l.com/"
}

tk.Label(left, text="URL", font=("blod", 25, "bold"), fg="#FF0303", bg=BG).pack(pady=5)
for name, url in web_apps.items():
    tk.Button(left, text=name, command=lambda u=url: webbrowser.open(u), bg="#222", fg="#FF0000", font=("bold", 12)).pack(fill="y", pady=3)

mac_apps = {
    "Terminal": "Terminal",
    "Documents": "Documents"
}

tk.Label(left, text="Apps", font=("bold", 20 , "bold"), fg="#FF0000", bg=BG).pack(pady=8)
for name, app in mac_apps.items():
    tk.Button(left, text=name, command=lambda a=app: os.system(f"open -a {a}"), bg="#222", fg="#FF0000", font=("bold", 12)).pack(fill="both", pady=3)

def add_web_app():
    name = simpledialog.askstring("Add Web App", "Name:")
    url = simpledialog.askstring("Add Web App", "URL:")
    if name and url:
        web_apps[name] = url
        messagebox.showinfo("Added", f"Added web app: {name}")

def add_mac_app():
    name = simpledialog.askstring("Add Mac App", "Name:")
    app = simpledialog.askstring("Add Mac App", "App Name:")
    if name and app:
        mac_apps[name] = app
        messagebox.showinfo("Added", f"Added mac app: {name}")

tk.Button(left, text="Add Web App", command=add_web_app, bg="#222", fg="#FF0000", font=("bold", 10)).pack(fill="x", pady=4)
tk.Button(left, text="Add Mac App", command=add_mac_app, bg="#222", fg="#FF0000", font=("bold", 10)).pack(fill="x", pady=4)

# ---------------- START the app ----------------
root.mainloop()
